# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import re

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.multioutput import MultiOutputClassifier




from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marcelot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marcelot\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marcelot\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine(r'sqlite:///../data/DisasterResponse.db')
df = pd.read_sql('SELECT * FROM classified_msgs', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
list(df.columns)

['id',
 'message',
 'original',
 'genre',
 'related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [4]:
X = df['message']
Y = df.iloc[:,4:]

In [5]:
list(Y.columns)

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    # prep nltk transformation objects
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    lemmed = [lemmatizer.lemmatize(word, pos='v') for word in tokens if word not in stop_words]
    
    # Reduce words to their stems
    stemmed = [stemmer.stem(word) for word in lemmed]

    return stemmed

Test the tokenize function:

In [7]:
for i in range(10):
    print(X[i])
    print(tokenize(X[i]))
    print("")
    print('--------------------------')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']

--------------------------
Is the Hurricane over or is it not over
['hurrican']

--------------------------
Looking for someone but no name
['look', 'someon', 'name']

--------------------------
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogan', '80', '90', 'destroy', 'hospit', 'st', 'croix', 'function', 'need', 'suppli', 'desper']

--------------------------
says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'countri', 'today', 'tonight']

--------------------------
Information about the National Palace-
['inform', 'nation', 'palac']

--------------------------
Storm at sacred heart of jesus
['storm', 'sacr', 'heart', 'jesu']

--------------------------
Please, we need tents and water. We are in Silo, Thank you!
['plea

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
        ('vect' , CountVectorizer(tokenizer=tokenize)),
        ('tfidf' , TfidfTransformer()),
        ('clf' , MultiOutputClassifier(AdaBoostClassifier(random_state = 42)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42)

In [10]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000237D296F0D8>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
Y_pred = pipeline.predict(X_test)

In [12]:
# first a general overview:
print(classification_report(Y_test, Y_pred,  target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.80      0.97      0.88      5001
               request       0.77      0.52      0.62      1093
                 offer       0.00      0.00      0.00        32
           aid_related       0.75      0.61      0.67      2700
          medical_help       0.58      0.25      0.35       532
      medical_products       0.64      0.32      0.43       345
     search_and_rescue       0.63      0.16      0.26       165
              security       0.27      0.05      0.08       127
              military       0.50      0.27      0.35       197
           child_alone       0.00      0.00      0.00         0
                 water       0.74      0.67      0.70       408
                  food       0.82      0.69      0.75       723
               shelter       0.74      0.56      0.64       590
              clothing       0.67      0.29      0.41        95
                 money       0.49      

C:\Users\marcelot\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\marcelot\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\marcelot\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\marcelot\Anaconda3\lib\site-packages\sklearn\

In [20]:
# now per class
acc = []
f1 = []
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:, i]))
    f1.append(f1_score(Y_test[col], Y_pred[:, i], average = 'macro'))
    acc.append(accuracy_score(Y_test[col], Y_pred[:, i]))
    
print("-------------------------------------------------------")    
print("Mean accuracy score: {:.4f}".format(np.mean(acc)))
print("Mean f1-score (macro): {:.4f}".format(np.mean(f1)))

related
              precision    recall  f1-score   support

           0       0.68      0.24      0.35      1553
           1       0.80      0.97      0.88      5001

    accuracy                           0.79      6554
   macro avg       0.74      0.60      0.62      6554
weighted avg       0.77      0.79      0.75      6554

request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5461
           1       0.77      0.52      0.62      1093

    accuracy                           0.89      6554
   macro avg       0.84      0.74      0.78      6554
weighted avg       0.89      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      655

earthquake
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      5924
           1       0.89      0.80      0.84       630

    accuracy                           0.97      6554
   macro avg       0.93      0.89      0.91      6554
weighted avg       0.97      0.97      0.97      6554

cold
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6432
           1       0.71      0.30      0.42       122

    accuracy                           0.98      6554
   macro avg       0.85      0.65      0.70      6554
weighted avg       0.98      0.98      0.98      6554

other_weather
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      6235
           1       0.44      0.16      0.24       319

    accuracy                           0.95      6554
   macro avg       0.70      0.58      0.61      6554
weighted avg       0.93      0.95      0.94 

In [14]:
#define styler
def is_equal(row):
    is_equal = row[0]==row[1]
    
    if is_equal:
        return ['background-color: lightgreen','background-color: lightgreen' ]
    else:
        return ['background-color: red','background-color: red'] 

# define function to compare actual with prediction:
def check_row(i):
    print(df['message'][i])
    print(tokenize(X[i]))
    check = pd.DataFrame(np.column_stack([Y_test.iloc[i,], Y_pred[i,]]), index = Y_test.columns, columns = ['Real','Pred'])
    return check.style.apply(is_equal, axis = 1)

In [15]:
check_row(1)

Is the Hurricane over or is it not over
['hurrican']


,Real,Pred
related,1,1
request,0,0
offer,0,0
aid_related,1,0
medical_help,0,0
medical_products,0,0
search_and_rescue,0,0
security,0,0
military,0,0
child_alone,0,0


In [16]:
check_row(20)

I would like to know if one of the radio ginen Journalist died?
['would', 'like', 'know', 'one', 'radio', 'ginen', 'journalist', 'die']


,Real,Pred
related,1,1
request,0,0
offer,0,0
aid_related,0,0
medical_help,0,0
medical_products,0,0
search_and_rescue,0,0
security,0,0
military,0,0
child_alone,0,0


In [17]:
check_row(1500)

Hello i would like to know if, even if you are not injured, can you go to the hospital for pounding heart. 
['hello', 'would', 'like', 'know', 'even', 'injur', 'go', 'hospit', 'pound', 'heart']


,Real,Pred
related,1,1
request,0,0
offer,0,0
aid_related,0,1
medical_help,0,0
medical_products,0,0
search_and_rescue,0,0
security,0,0
military,0,0
child_alone,0,0


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.